In [ ]:
import imageio as iio
from matplotlib import pyplot as plt
import numpy as np
import cv2
import imutils
from PIL import Image, ImageDraw
import itertools
from scipy.ndimage.filters import gaussian_filter as __gaussian_filter
from scipy.ndimage.filters import convolve as __convolve
from scipy.ndimage.filters import correlate as __correlate
from scipy.fftpack import fftshift as __fftshift

import nibabel as nib
import pandas as pd

<ipython-input-1-0882c34063c9>:8: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter as __gaussian_filter
<ipython-input-1-0882c34063c9>:9: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import convolve as __convolve
<ipython-input-1-0882c34063c9>:10: DeprecationWarning: Please use `correlate` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import correlate as __correlate


In [ ]:
def process_image(img):



    def calculate_image_centroid(image):
        # Convert the image to grayscale
        grayscale_image = np.array(image.convert("L"))


        # Get the height and width of the image
        height, width = grayscale_image.shape

        # Initialize variables for weighted sums and total intensity
        weighted_sum_x = 0
        weighted_sum_y = 0
        total_intensity = 0

        # Iterate through each pixel
        for y in range(height):
            for x in range(width):
                intensity = grayscale_image[y, x]

                # Calculate the weighted sum of pixel coordinates
                weighted_sum_x += intensity * x
                weighted_sum_y += intensity * y

                # Calculate the sum of pixel intensities
                total_intensity += intensity

        # Calculate the centroid coordinates
        centroid_x = weighted_sum_x / total_intensity
        centroid_y = weighted_sum_y / total_intensity

        return centroid_x, centroid_y

    # Convert the image to PIL Image
    pil_image = Image.fromarray(img)

    centroid = calculate_image_centroid(pil_image)

    # cambiar el centro de la imagen
    def change_image_center(img, new_center):
        height, width = img.shape[:2]
        center_x, center_y = new_center
        current_center_point = (width // 2, height // 2)

        # Calculate the pixel shifts for padding
        if current_center_point[0] == center_x and current_center_point[1] == center_y:
          padded_img = img
        if current_center_point[0] > center_x and current_center_point[1]> center_y:
          padded_img = cv2.copyMakeBorder(img, int(height-2*center_y), 0, 0,0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        if current_center_point[0] == center_x and current_center_point[1]> center_y:
          padded_img = cv2.copyMakeBorder(img, int(height-2*center_y), 0, int(width-2*center_x),0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        if current_center_point[0] < center_x and current_center_point[1]>center_y:
          padded_img = cv2.copyMakeBorder(img, int(height-2*center_y), 0, 0, int(width-2*(width-center_x)), cv2.BORDER_CONSTANT, value=(0, 0, 0))
        if current_center_point[0] < center_x and current_center_point[1]==center_y:
          padded_img = cv2.copyMakeBorder(img, 0, 0, 0, int(width-2*(width-center_x)), cv2.BORDER_CONSTANT, value=(0, 0, 0))
        if current_center_point[0] > center_x and current_center_point[1] < center_y:
          padded_img = cv2.copyMakeBorder(img, 0, int(height-2*(height-center_y)), int(width-2*center_x), 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        if current_center_point[0] > center_x and current_center_point[1] == center_y:
          padded_img = cv2.copyMakeBorder(img, 0, 0, int(width-2*center_x), 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        if current_center_point[0] < center_x and current_center_point[1] < center_y:
          padded_img = cv2.copyMakeBorder(img, 0, int(height-2*(height-center_y)), 0, int(width-2*(width-center_x)), cv2.BORDER_CONSTANT, value=(0, 0, 0))
        if current_center_point[0] == center_x and current_center_point[1] < center_y:
          padded_img = cv2.copyMakeBorder(img, 0, int(height-2*(height-center_y)), 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))

        return padded_img

    img2a = change_image_center(img, centroid)

    # Agrego margen
    m=5
    img2 = cv2.copyMakeBorder(img2a, m, m, m, m, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    def rotate_and_flip_image(img, angles, centers):
        rotated_and_flipped_images = []

        for angle, center in itertools.product(angles, centers):
            # change_image_center
            padded_img=change_image_center(img, center)
            height, width = padded_img.shape[:2]
            centerX, centerY = (width // 2, height // 2)
            # get rotation matrix with the specified center
            M = cv2.getRotationMatrix2D((centerX,centerY), angle, 1.0)
            rotated = cv2.warpAffine(padded_img, M, (width, height))


            flipped = np.fliplr(rotated)

            rotated_and_flipped_images.append((rotated, flipped))

        return rotated_and_flipped_images

    angles = [0, 0.5, -0.5, 1, -1, 1.5, -1.5, 2, -2, 2.5, -2.5, 3, -3, 3.5, -3.5, 4, -4, 4.5, -4.5, 5, -5]  # Example angles, you can modify this list as per your requirements
    height, width = img2.shape[:2]
    centerx, centery = (width // 2, height // 2)
    #centerx=cropped_image2.shape[1]/2
    #centery=cropped_image2.shape[0]/2
    b=1 #cuanto mover el centro?
    d=2
    centers = [(centerx,centery), (centerx+b,centery), (centerx-b,centery), (centerx,centery-b), (centerx,centery+b),(centerx-b,centery+b),(centerx-b,centery-b),(centerx+b,centery+b),(centerx+b,centery-b),(centerx+d,centery), (centerx-d,centery), (centerx,centery-d), (centerx,centery+d),(centerx-d,centery+d),(centerx-d,centery-d),(centerx+d,centery+d),(centerx+d,centery-d)]
    result = rotate_and_flip_image(img2, angles, centers)

    def __is_grey_scale(image):
        """
        Check whether an image is grayscale or not
        :param image: image data
        :return: True if the image is grayscale, False otherwise
        """
        if len(image.shape) > 2:
            return False
        else:
            return True

    def gms(reference, query, pool='mean'):
        """
        Computes the Gradient Magnitude Similarity Mean (GMSM) and the Gradient Magnitude Similarity Deviation (GMSD)
        It also calculates the quality_map: local quality map of the distorted image and gradient map of each image
        GMSD is an implementation of the following algorithm: Wufeng Xue, Lei Zhang, Xuanqin Mou, and Alan C. Bovik,
        "Gradient Magnitude Similarity Deviation: A Highly Efficient Perceptual Image Quality Index",
        This code is a translation of the matlab code that can be downloaded here:
        http://www.comp.polyu.edu.hk/~cslzhang/IQA/GMSD/GMSD.htm
        :param reference: the reference image (grayscale image, double type, 0~255)
        :param query: the distorted image (grayscale image, double type, 0~255)
        :param pool: pooling strategy ('mean' for GMSM, 'std' for GMSD)
        :return: GMSM value if pool='mean', GMSD value if pool='std'
        """

        if not(__is_grey_scale(query)):
            reference = cv2.cvtColor(reference, cv2.COLOR_BGR2GRAY) # convert to gray scale
            query = cv2.cvtColor(query, cv2.COLOR_BGR2GRAY)

        (C, Down_step) = (170, 2)

        hx = np.array([[1.0, 0.0, -1.0], [1.0, 0.0, -1.0], [1.0, 0.0, -1.0]]) / 3
        hy = hx.transpose()
        # these are the Prewitt filters along horizontal (x) and vertical (y) directions

        (ref, dist) = (reference.astype('double'), query.astype('double'))

        aveKernel = np.array([[1.0, 1.0], [1.0, 1.0]]) / 4  # averaging filter

        averef = __convolve(ref, aveKernel, mode='constant', cval=0.0)
        avedist = __convolve(dist, aveKernel, mode='constant', cval=0.0)

        ref = averef[::Down_step, ::Down_step]
        dist = avedist[::Down_step, ::Down_step]

        Ixref = __convolve(ref, hx, mode='constant', cval=0.0)
        Iyref = __convolve(ref, hy, mode='constant', cval=0.0)
        gradientMap1 = np.sqrt(Ixref**2 + Iyref**2)

        Ixdist = __convolve(dist, hx, mode='constant', cval=0.0)
        Iydist = __convolve(dist, hy, mode='constant', cval=0.0)
        gradientMap2 = np.sqrt(Ixdist**2 + Iydist**2)

        quality_map = (2 * gradientMap1 * gradientMap2 + C) / (gradientMap1**2 + gradientMap2**2 + C)

        if pool == 'mean':
            return np.mean(quality_map)
        elif pool == 'std':
            return np.std(quality_map)

    # n es la cantidad de ángulos que se prueban
    n=len(angles)
    m=len(centers)

    def similaridad(lista):
        #img = cv2.imread(image_path)  # Read the image
        similaridad = []

        for i in range(n*m):
            # Aplico medidad de similitud
            med=gms(lista[i][0] ,lista[i][1], pool='std')


            # Append rotated and flipped image to the list
            similaridad.append(med)

        return similaridad

    sim = similaridad(result)
    sim

    # Calculamos el mínimo

    minimum_value = min(sim)
    minimum_index = sim.index(minimum_value)
    print(minimum_value)
    print(minimum_index)

    v=(minimum_index,minimum_value)

    return v


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# List of image paths CONTROL
image_paths = ["/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0001_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0002_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0010_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0011_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0013_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0018_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0019_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0020_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0026_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0030_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0032_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0033_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0034_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0044_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0058_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0062_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0064_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0065_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0068_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0069_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0070_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0071_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0072_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0074_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0075_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0078_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0083_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0085_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0086_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0096_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0106_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0109_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0110_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0112_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0113_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0114_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0116_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0130_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0133_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0135_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0138_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0139_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0146_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0157_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0165_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0169_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0170_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0176_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0177_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0180_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0181_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0186_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0188_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0195_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0197_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0199_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0200_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0201_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0203_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0204_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0206_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0207_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0208_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0212_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0216_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0220_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0221_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0228_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0229_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0234_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0237_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0241_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0244_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0254_MR1_mpr_n6_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0255_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0256_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0259_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0260_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0262_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0266_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0270_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0271_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0274_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0279_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0280_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0284_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0289_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0292_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0293_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0299_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0301_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0303_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0309_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0317_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0322_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0323_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0326_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0330_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0332_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0337_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0338_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0341_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0342_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0343_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0345_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0354_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0355_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0356_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0357_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0358_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0363_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0365_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0366_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0367_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0369_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0371_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0372_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0378_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0381_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0398_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0401_MR1_mpr_n5_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0404_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0422_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0423_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0426_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0428_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0433_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0438_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0443_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0445_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0446_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0449_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0455_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0456_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0457_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               ]




# List to store the processed matrix
processed_matrixs_CONTROL = []

# Process each image
for image_path in image_paths:
    # Open the image
    img = nib.load(image_path)

    # Access the image data as a NumPy array
    image_data = img.get_fdata()

    image_data = image_data.squeeze()


    # elijo slice para ver, [59,64,69,74,79,84,89,94,99,104,109,114,119,124,129,134,139,144,149]
    processed_vectors_CONTROL = []
    for i in [39,44,49,54,154,159,164,169]:
        img90=image_data[:,i,:]
        image2 = imutils.rotate_bound(img90, angle=-90)

    # image2 = np.diagonal(image_data, offset=0, axis1=1, axis2=2).T #oblicua

        image = cv2.convertScaleAbs(image2, alpha=0.03)

    # Process the image
        processed_vector_CONTROL = process_image(image)

        processed_vectors_CONTROL.append(processed_vector_CONTROL)

    processed_matrix_CONTROL = processed_vectors_CONTROL

    processed_matrixs_CONTROL.append(processed_matrix_CONTROL)

0.06877831838807083
56
0.07230080856205927
65
0.07169054706984948
92
0.0786510367871807
130
0.06916608644975569
58
0.07053233335689657
59
0.05805463256020404
62
0.0491237878219574
122
0.0544053842730794
62
0.060518290360949974
59
0.072192127339469
58
0.0712075877930319
62
0.067561502982797
56
0.06329274067826662
56
0.05737616471379721
25
0.056939319089941126
25
0.08805394097329729
56
0.08442389386632089
59
0.09128638113197447
86
0.08751768247048104
63
0.08092636989690714
62
0.08775667783650155
65
0.08262922204593492
127
0.05778003721254317
99
0.06924457351827701
62
0.0762976285311367
130
0.08213748994989721
63
0.08318653758297644
62
0.0763686709231778
90
0.07323825473967019
90
0.06254841150428472
63
0.05511462498148383
169
0.06800350864660618
62
0.08089913695869384
97
0.07693877774308856
165
0.08683514210523292
97
0.08187224077972154
56
0.07388172641772611
63
0.06854213247787948
62
0.05948007922418094
59
0.06027044408012537
63
0.062474808981022714
63
0.0726018578337577
28
0.06805381994

In [ ]:
processed_matrixs_CONTROL

[[(56, 0.06877831838807083),
  (65, 0.07230080856205927),
  (92, 0.07169054706984948),
  (130, 0.0786510367871807),
  (58, 0.06916608644975569),
  (59, 0.07053233335689657),
  (62, 0.05805463256020404),
  (122, 0.0491237878219574)],
 [(62, 0.0544053842730794),
  (59, 0.060518290360949974),
  (58, 0.072192127339469),
  (62, 0.0712075877930319),
  (56, 0.067561502982797),
  (56, 0.06329274067826662),
  (25, 0.05737616471379721),
  (25, 0.056939319089941126)],
 [(56, 0.08805394097329729),
  (59, 0.08442389386632089),
  (86, 0.09128638113197447),
  (63, 0.08751768247048104),
  (62, 0.08092636989690714),
  (65, 0.08775667783650155),
  (127, 0.08262922204593492),
  (99, 0.05778003721254317)],
 [(62, 0.06924457351827701),
  (130, 0.0762976285311367),
  (63, 0.08213748994989721),
  (62, 0.08318653758297644),
  (90, 0.0763686709231778),
  (90, 0.07323825473967019),
  (63, 0.06254841150428472),
  (169, 0.05511462498148383)],
 [(62, 0.06800350864660618),
  (97, 0.08089913695869384),
  (165, 0.076

In [ ]:
rows = []
for sublist in processed_matrixs_CONTROL:
    #row_dict = {'Column1': sublist[0][1], 'Column2': sublist[1][1], 'Column3': sublist[2][1], 'Column4': sublist[3][1],'Column5': sublist[4][1], 'Column6': sublist[5][1],'Column7': sublist[6][1], 'Column8': sublist[7][1],'Column9': sublist[8][1], 'Column10': sublist[9][1],'Column11': sublist[10][1], 'Column12': sublist[11][1],'Column13': sublist[12][1], 'Column14': sublist[13][1],'Column15': sublist[14][1], 'Column16': sublist[15][1],'Column17': sublist[16][1], 'Column18': sublist[17][1],'Column19': sublist[18][1]}
    row_dict = {'Column1': sublist[0][1], 'Column2': sublist[1][1], 'Column3': sublist[2][1], 'Column4': sublist[3][1],'Column5': sublist[4][1], 'Column6': sublist[5][1],'Column7': sublist[6][1], 'Column8': sublist[7][1]}
    rows.append(row_dict)

# Convertir la lista de diccionarios a un DataFrame
control = pd.DataFrame(rows)

# Mostrar el DataFrame
print(control)


      Column1   Column2   Column3   Column4   Column5   Column6   Column7  \
0    0.068778  0.072301  0.071691  0.078651  0.069166  0.070532  0.058055   
1    0.054405  0.060518  0.072192  0.071208  0.067562  0.063293  0.057376   
2    0.088054  0.084424  0.091286  0.087518  0.080926  0.087757  0.082629   
3    0.069245  0.076298  0.082137  0.083187  0.076369  0.073238  0.062548   
4    0.068004  0.080899  0.076939  0.086835  0.081872  0.073882  0.068542   
..        ...       ...       ...       ...       ...       ...       ...   
130  0.064955  0.073420  0.076267  0.078708  0.074990  0.072027  0.070626   
131  0.052228  0.056562  0.055584  0.061385  0.065636  0.062699  0.057975   
132  0.069744  0.071954  0.067366  0.071567  0.063262  0.064420  0.058767   
133  0.075271  0.075831  0.086682  0.090494  0.076712  0.081793  0.068855   
134  0.067451  0.074630  0.075576  0.082846  0.067740  0.069763  0.060612   

      Column8  
0    0.049124  
1    0.056939  
2    0.057780  
3    0.0551

In [ ]:
# List of image paths AD
image_paths = ["/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0003_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0015_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0016_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0021_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0022_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0023_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0028_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0031_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0035_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0039_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0041_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0042_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0046_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0052_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0053_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0056_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0060_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0066_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0067_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0073_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0082_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0084_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0094_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0098_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0115_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0120_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0122_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0123_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0124_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0134_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0137_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0142_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0143_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0155_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0158_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0161_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0164_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0166_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0179_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0184_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0185_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0205_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0210_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0217_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0223_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0226_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0233_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0238_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0240_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0243_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0247_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0263_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0267_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0268_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0269_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0272_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0273_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0278_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0286_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0287_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0288_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0290_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0291_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0298_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0300_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0304_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0307_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0308_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0312_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0315_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0316_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0329_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0335_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0339_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0351_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0352_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0362_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0373_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0374_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0380_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0382_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0388_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0390_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0399_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0400_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0402_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0405_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0411_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0418_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0424_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0425_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0430_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0432_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0440_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0441_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0447_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0451_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0452_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0453_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0454_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               ]




# List to store the processed vectors
processed_matrixs_AD = []

# Process each image
for image_path in image_paths:
    # Open the image
    img = nib.load(image_path)

    # Access the image data as a NumPy array
    image_data = img.get_fdata()

    image_data = image_data.squeeze()
    # elijo slice para ver, por ejemplo la 90 [59,64,69,74,79,84,89,94,99,104,109,114,119,124,129,134,139,144,149]
    processed_vectors_AD = []
    for i in [39,44,49,54,154,159,164,169]:
        img90=image_data[:,i,:]
        image2 = imutils.rotate_bound(img90, angle=-90)

       # image2 = np.diagonal(image_data, offset=0, axis1=1, axis2=2).T  # oblicua

        image = cv2.convertScaleAbs(image2, alpha=0.03)

    # Process the image
        processed_vector_AD = process_image(image)

        processed_vectors_AD.append(processed_vector_AD)

    processed_matrix_AD = processed_vectors_AD

    processed_matrixs_AD.append(processed_matrix_AD)

0.07735369822545787
25
0.0910620971360257
59
0.08870954280776995
62
0.0846702878273812
58
0.06529168451079874
58
0.06474199029950584
62
0.06552755787196828
58
0.05788572050925537
67
0.07185241635343366
29
0.07657387835936964
62
0.07840379948840182
29
0.08646691723186366
46
0.08063704145776117
56
0.06845356447016691
25
0.06753457183978535
24
0.05874886329111295
93
0.06009365517616629
59
0.05750180617181988
63
0.06833063815412975
63
0.07411726089081011
92
0.06493205630858599
130
0.06837152476938903
56
0.06205414854726561
24
0.05265353535308655
92
0.06391317455395176
133
0.07303339768685445
92
0.08187120445012336
63
0.0875618693926376
58
0.0733635852366278
56
0.06633023295267448
58
0.071446229854111
67
0.056870209503359664
59
0.05815776473391301
127
0.07487746622116172
133
0.07131054387411276
58
0.07369275824926956
96
0.07383226613139321
56
0.07670541224667575
130
0.07011398707234934
25
0.06347534675080412
59
0.0782611501760194
59
0.08379718948251708
63
0.07933285792929158
63
0.0782151084

In [ ]:
processed_matrixs_AD

[[(25, 0.07735369822545787),
  (59, 0.0910620971360257),
  (62, 0.08870954280776995),
  (58, 0.0846702878273812),
  (58, 0.06529168451079874),
  (62, 0.06474199029950584),
  (58, 0.06552755787196828),
  (67, 0.05788572050925537)],
 [(29, 0.07185241635343366),
  (62, 0.07657387835936964),
  (29, 0.07840379948840182),
  (46, 0.08646691723186366),
  (56, 0.08063704145776117),
  (25, 0.06845356447016691),
  (24, 0.06753457183978535),
  (93, 0.05874886329111295)],
 [(59, 0.06009365517616629),
  (63, 0.05750180617181988),
  (63, 0.06833063815412975),
  (92, 0.07411726089081011),
  (130, 0.06493205630858599),
  (56, 0.06837152476938903),
  (24, 0.06205414854726561),
  (92, 0.05265353535308655)],
 [(133, 0.06391317455395176),
  (92, 0.07303339768685445),
  (63, 0.08187120445012336),
  (58, 0.0875618693926376),
  (56, 0.0733635852366278),
  (58, 0.06633023295267448),
  (67, 0.071446229854111),
  (59, 0.056870209503359664)],
 [(127, 0.05815776473391301),
  (133, 0.07487746622116172),
  (58, 0.07

In [ ]:
rows2 = []
for sublist in processed_matrixs_AD:
    #row2_dict = {'Column1': sublist[0][1], 'Column2': sublist[1][1], 'Column3': sublist[2][1], 'Column4': sublist[3][1],'Column5': sublist[4][1], 'Column6': sublist[5][1],'Column7': sublist[6][1], 'Column8': sublist[7][1],'Column9': sublist[8][1], 'Column10': sublist[9][1],'Column11': sublist[10][1], 'Column12': sublist[11][1],'Column13': sublist[12][1], 'Column14': sublist[13][1],'Column15': sublist[14][1], 'Column16': sublist[15][1],'Column17': sublist[16][1], 'Column18': sublist[17][1],'Column19': sublist[18][1]}
    row2_dict = {'Column1': sublist[0][1], 'Column2': sublist[1][1], 'Column3': sublist[2][1], 'Column4': sublist[3][1],'Column5': sublist[4][1], 'Column6': sublist[5][1],'Column7': sublist[6][1], 'Column8': sublist[7][1]}
    rows2.append(row2_dict)

# Convertir la lista de diccionarios a un DataFrame
ad = pd.DataFrame(rows2)

# Mostrar el DataFrame
print(ad)

     Column1   Column2   Column3   Column4   Column5   Column6   Column7  \
0   0.077354  0.091062  0.088710  0.084670  0.065292  0.064742  0.065528   
1   0.071852  0.076574  0.078404  0.086467  0.080637  0.068454  0.067535   
2   0.060094  0.057502  0.068331  0.074117  0.064932  0.068372  0.062054   
3   0.063913  0.073033  0.081871  0.087562  0.073364  0.066330  0.071446   
4   0.058158  0.074877  0.071311  0.073693  0.073832  0.076705  0.070114   
..       ...       ...       ...       ...       ...       ...       ...   
95  0.067555  0.070971  0.079059  0.082198  0.071234  0.066760  0.065999   
96  0.070674  0.086206  0.087437  0.089444  0.071196  0.064831  0.066556   
97  0.068323  0.068036  0.074806  0.084008  0.072977  0.061191  0.061178   
98  0.057550  0.067953  0.076277  0.080116  0.073234  0.070945  0.062026   
99  0.071723  0.066431  0.079558  0.078543  0.065274  0.062340  0.063175   

     Column8  
0   0.057886  
1   0.058749  
2   0.052654  
3   0.056870  
4   0.063475

In [ ]:
from google.colab import files
# Export to CSV
control.to_csv('control.csv')
# Download the file
files.download('control.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Export to CSV
ad.to_csv('ad.csv')
# Download the file
files.download('ad.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>